# Chapter 3: Effective Theory of Deep Linear Networks at Initialization

This document will explore the claims laid out in Chapter 3 of Principles of Deep Learning Theory (it will be referred to from hereon as PDLT).
In particular, it discusses the distribution of variables contained within deep linear networks with no bias. After some mathematical analysis, it claims the following:

If the parameters in the weights &c are initialised according to Gaussian distributions, then the following should be true:
- The first layer has a distribution which is Gaussian
- Subsequent layers deviate from Gaussian, and this deviance has a scale that is related to the depth-to-width ratio.
    - If this ratio if small, its deviation from Gaussianity is linear in the ratio (if we look at the 4-point connected correlator (also known as cumulant) of the preactivation distribution)
    - If the ratio is large, its equivalent connected correlator varies exponentially from Gaussianity

It also shows that in the limit of infinite width, the neuron values become Gaussian, which implies that the contributions for a neuron from those in the previous layer cancel out.
The text claims that riding this line of complete order (where no deviance from a purely random Gaussian distribution) and chaos (where the network is so inconsistent between instantiations, and there is a lot of noise between neurons) is what defines the ability of a network to learn sufficiently complex non-linear functions.

Reading the text will hopefully help to illuminate this claim; however, this notebook will attempt to verify their claim of the above using neural networks created with Pytorch. It's worth noting that their technical definition will vary from a normal attempt of creating a deep linear network in Pytorch, due to the fact that we will need to keep track of the values of the correlators (in this case, the expectations of different monomials of the network preactivations).

## Defining the deep learning model with access to preactivations for statistical analysis

First, we need to define a neural network which is a deep linear model and which will also allow us to be able to read out the values of its preactivations at each layer. These are not usually easily accessible when creating a model, so we will have to edit the flow to make sure these can be accessed and saved.

For now, to make the maths easier, we will set the input dimension to 10, all layers to the same width of 10, and the model depth shall be 10 as well.

For the initialisation of each layer, equation (3.4) in §3 states the following choice of mean and standard deviation of the weights in the model.

More specifically, for a deep linear model with max depth $L$, given the preactivations $z_i^{(\ell)}$ for layer $\ell$, and the weights $W_{ij}$ are such that $z_i^{(\ell + 1)} = W_{ij}^{(\ell + 1)} z_j^{(\ell)}$, then the initialisation distribution for $W_{ij}^{(\ell)}$ are as follows:

$$\begin{aligned}
\mathbb{E}[W_{ij}^{(\ell)}] &= 0 \\
\mathbb{E}[W_{i_1 j_1}^{(\ell)} W_{i_2 j_2}^{(\ell)}] &= \delta_{i_1 i_2} \delta_{j_1 j_2} \frac{C_W}{n_{\ell - 1}}
\end{aligned}$$

The variance is normalised as such as this factor then cancels out when integrating over the previous layer.

So, we shall define the variables used in the model as follows:

$$\begin{aligned}
L &= 10 \\
n_{\ell} &= 10 \quad \forall \, \ell \\
C_W &= 1
\end{aligned}$$

First, Python imports need to be laid out:

In [84]:
%matplotlib inline

from functools import reduce

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch import nn

In [85]:
from dataclasses import dataclass

from typing import Tuple
from torch import Tensor
from torch.nn import Linear

MODEL_DEPTH = 10
LAYER_WIDTH = 10

C_W = 1.0


@dataclass
class Metadata:
    """
    This is a dataclass which is to store statistical metadata about
    preactivations in a layer.

    The list of metadata collected from a layer is as follows:
    - 2-point correlators for different nodes and same nodes
    - 4-point correlators for different nodes and same nodes
    """
    # 2-point correlators, both for separate nodes and for the same nodes
    z11: float  # 2-point correlator, different nodes
    z12: float  # 2-point correlator, same nodes

    # 4-point correlators, both for separate nodes and for the same nodes
    z1234: float  # 4-point correlator, 4 different nodes
    z1123: float  # 4-point correlator, 3 different nodes
    z1122: float  # 4-point correlator, 2 different nodes


LayerApplicationAccumulator = Tuple[Tensor, list[Metadata]]


def get_statistical_metadata(preactivations: Tensor) -> Metadata:
    """
    Generate the metadata for preactivations.
    Return the dataclass defined above which contains what's needed.

    Args:
        preactivations: Tensor
            The input for a layer
    
    Returns:
        Dataclass storing all of the metadata of interest.
    """
    return Metadata(
        # Mix and match which indices are used to try and minimise
        # statistical correlation between nodes
        z11=(preactivations[4]*preactivations[4]).item(),
        z12=(preactivations[1]*preactivations[2]).item(),
        z1234=(preactivations[3]*preactivations[4]*preactivations[5]*preactivations[6]).item(),
        z1123=(preactivations[1]*preactivations[1]*preactivations[2]*preactivations[3]).item(),
        z1122=(preactivations[4]*preactivations[4]*preactivations[7]*preactivations[7]).item(),
    )


def layer_fold_with_metadata(accumulator: LayerApplicationAccumulator, layer: Linear) -> LayerApplicationAccumulator:
    layer_input, metadata = accumulator
    layer_output = layer(layer_input)

    return layer_output, [*metadata, get_statistical_metadata(layer_output)]


class DeepLinearModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.layers = [
            self.get_normal_initialised_layer()
            for _ in range(MODEL_DEPTH)
        ]

        self.metadata = []

    @staticmethod
    def get_normal_initialised_layer() -> nn.Linear:
        layer = nn.Linear(LAYER_WIDTH, LAYER_WIDTH, bias=False)
        # sqrt is added because the defined value in the literature is the variance,
        # not the sqrt
        nn.init.normal_(layer.weight, mean=0.0, std=np.sqrt(C_W/LAYER_WIDTH))

        return layer
    
    def forward(self, model_input):
        model_output, metadata = list(reduce(layer_fold_with_metadata, self.layers, (model_input, [])))
        self.metadata = metadata

        return model_output

With the model defined, the next step is to randomly generate points with a model and retrieve the preactivations from it.
The aim is then to see the generated distributions and see if their values match what was predicted theoretically, in particular in §3.3.

In [86]:
# Theoretically, the distribution of the input should not matter, so
# we will just generate a simple 10-dimensional input for these networks
MODEL_INPUT = 3*torch.rand(LAYER_WIDTH)

def generate_preactivations():
    model = DeepLinearModel()
    _ = model(MODEL_INPUT)
    metadata_list = model.metadata

    return np.array([
        [
            # 2-point correlators, both for separate nodes and for the same nodes
            layer_data.z11,
            layer_data.z12,

            # 4-point correlators, both for separate nodes and for the same nodes
            layer_data.z1122,
            layer_data.z1123,
            layer_data.z1234,
        ]
        for layer_data in metadata_list
    ])


SAMPLE_COUNT = 5_000

# preallocate space in memory to speed things up
preactivation_data = np.empty(shape=(SAMPLE_COUNT, MODEL_DEPTH, 5), dtype=float)
with torch.no_grad():
    for idx in range(SAMPLE_COUNT):
        preactivation_data[idx] = generate_preactivations()

First, given we now have this data, it would help data analysis if the hierarchy were changed in the indexing, ie. if the first index referred to each of the different data types

In [87]:
transposed_preactivations = preactivation_data.T

z11_data = transposed_preactivations[0]
z12_data = transposed_preactivations[1]
z1122_data = transposed_preactivations[2]
z1123_data = transposed_preactivations[3]
z1234_data = transposed_preactivations[4]

Now that we have organised the data, it is now time to analyse it

## Layer 1

### 2-point correlators

Theoretically, this is defined in equation (3.9), ie.

$$\mathbb{E}[z_i^{(1)}z_j^{(1)}] = \delta_{ij}C_W G^{(0)}$$

where $G^{(0)}$ is defined as $\frac{1}{n_0} \sum_{i} (x_i)^2$.

So, for separate nodes ($i \ne j$), $\mathbb{E}[z_i^{(1)}z_j^{(1)}] = 0$.
This corresponds to `z12_data`.

On the other hand, `z11_data` refers to when $i = j$, so we will have to verify this against the experimental value of $G^{(0)}$.

In [88]:
z12_layer_1 = z12_data[0]
z11_layer_1 = z11_data[0]

z12_layer_1_mean = np.mean(z12_layer_1)
z11_layer_1_mean = np.mean(z11_layer_1)

# In this case, n_0 is the size of the input, or the layer width.
# G_0 here can be interpreted as the expectation of the square of the values in x
G_0 = np.sum((MODEL_INPUT**2).numpy()) / LAYER_WIDTH

expected_z11_layer_1_mean = C_W * G_0
expected_z12_layer_1_mean = 0.0

print('-------------------------------------------------------------------')
print('Layer 1:')
print(f'i == j - expected = {expected_z11_layer_1_mean:.5f}, observed = {z11_layer_1_mean:.5f}')
print(f'i != j - expected = {expected_z12_layer_1_mean:.5f}, observed = {z12_layer_1_mean:.5f}')

-------------------------------------------------------------------
Layer 1:
i == j - expected = 8.18355, observed = 8.31534
i != j - expected = 0.00000, observed = -0.23237
